In [ ]:
import cv2
import os
import math
import numpy as np

input_folder = "videos_input" 
output_folder = "videos_output"
num_frames_per_clip = 16  # nombre de frames par sous-vidéo

os.makedirs(output_folder, exist_ok=True)

# Parcourir toutes les vidéos du dossier
for video_name in os.listdir(input_folder):
    if not video_name.lower().endswith((".mp4", ".avi", ".mov")):
        continue

    video_path = os.path.join(input_folder, video_name)
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    #  Test si vidéo RGB ou IR
    ret, frame = cap.read()  
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # replacer sur la première frame
    dim = len(frame.shape) # si dim == 2: vidéo IR il faut mimer un pseudo-RGB

    frames = []
    clip_idx = 0

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        # Si IR / grayscale, convertir en pseudo-RGB
        if dim == 2:
            frame = np.stack([frame]*3, axis=-1)

        frames.append(frame)

        # Quand on a num_frames_per_clip frames, on sauvegarde
        if len(frames) == num_frames_per_clip:
            output_name = f"{os.path.splitext(video_name)[0]}_clip{clip_idx}.mp4"
            output_path = os.path.join(output_folder, output_name)
            out = cv2.VideoWriter(output_path,
                                  cv2.VideoWriter_fourcc(*"mp4v"),
                                  fps, (width, height))
            for f in frames:
                out.write(f)
            out.release()
            frames = []
            clip_idx += 1

    # Sauvegarder les frames restantes
    if frames:
        output_name = f"{os.path.splitext(video_name)[0]}_clip{clip_idx}.mp4"
        output_path = os.path.join(output_folder, output_name)
        out = cv2.VideoWriter(output_path,
                              cv2.VideoWriter_fourcc(*"mp4v"),
                              fps, (width, height))
        for f in frames:
            out.write(f)
        out.release()

    cap.release()
    print(f"{video_name} découpée en {clip_idx + 1} clips.")
